# Experiment 2: Two stage suborbital launcher


In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib widget

In [2]:
from dataclasses import dataclass
from math import radians
import random
import time

import numpy as np
import matplotlib.pyplot as plt
from tqdm.auto import trange

from cw.context import time_it
from cw.simulation import Simulation, StatesBase, AB3Integrator, BatchLogger, Logging

from topone.sim_post_processing import sim_post_processing
from topone.environment import Environment, Stage
from topone.pid_agent import PIDAgent

from agent import Agent
from ideal_agent import IdealAgent

## Simulation configuration

In [3]:
@dataclass
class States(StatesBase):
    t: float = 0
    command_engine_on: bool = False
    command_drop_stage: bool = False
    gii: np.ndarray = np.zeros(2)
    xii: np.ndarray = np.zeros(2)
    vii: np.ndarray = np.zeros(2)
    aii: np.ndarray = np.zeros(2)
    fii_thrust: np.ndarray = np.zeros(2)
    theta: float = 0.
    theta_dot: float = 0.
    mass: float = 0.
    mass_dot: float = 0.
    h: float = 0.
    engine_on: bool = False
    stage_state: int = 0
    stage_idx: int = 0
    gamma_i: float = 0.
    gamma_e: float = 0.
    latitude: float = 0.
        
    reward: float = 0.
    reward_integral: float = 0.
    delta_v: float = 0.

    def get_y_dot(self):
        y = np.empty(7)
        y[:2] = self.vii
        y[2:4] = self.aii
        y[4] = self.theta_dot
        y[5] = self.mass_dot
        y[6] = self.reward
        return y

    def get_y(self):
        y = np.empty(7)
        y[:2] = self.xii
        y[2:4] = self.vii
        y[4] = self.theta
        y[5] = self.mass
        y[6] = self.reward_integral
        return y

    def set_t_y(self, t, y):
        self.t = t
        self.xii = y[:2]
        self.vii = y[2:4]
        self.theta = y[4]
        self.mass = y[5]
        self.reward_integral = y[6]

In [25]:
agent = Agent(
    epsilon=0.01,
    alpha=0.9,
    gamma=0.9,
    path="./set_3"
)

# agent = IdealAgent()

simulation = Simulation(
    states_class=States,
    integrator=AB3Integrator(
        h=0.1,
        rk4=False,
        fd_max_order=1),
    modules=[
        Environment(
            surface_diameter=1737.4e3,
            mu=4.9048695e12,
            stages=(
                Stage(
                    dry_mass=2,
                    propellant_mass=0.02,
                    specific_impulse=100,
                    thrust=4*1.7),
                Stage(
                    dry_mass=1,
                    propellant_mass=0.02,
                    specific_impulse=100,
                    thrust=2*1.7),
            ),
            initial_altitude=1000,
            initial_theta_e=radians(90),
            initial_latitude=radians(90),
            reward_function=lambda s: s.h - 1000,
        ),
        agent
    ],
    logging=Logging(),
    initial_state_values=None,
)
batch_logger = BatchLogger()
batch_logger.initialize(simulation)
simulation.stash_states()

In [21]:
def post_processing(result):
    sim_post_processing(result)

## Batch run

In [7]:
def run_batch(n_episodes, backup_period=30):
    batch_logger.reset_batch()

    # Backup original logger and swap with faster logger
    original_logger = simulation.logging
    simulation.logging = batch_logger

    last_backup_time = time.time()

    try:
        for i in trange(n_episodes):
            simulation.restore_states()
#             agent.greedy_time = random.uniform(0, 70)
            simulation.run(100)
            if time.time() - last_backup_time >= backup_period:
                agent.store()
                last_backup_time = time.time()
    finally:
        agent.store()
        simulation.logging = original_logger
        batch_results = batch_logger.finish_batch()
        return batch_results

In [26]:
n_per_min = 8400
batch_results = run_batch(n_per_min * 60 * 5, backup_period=60*30)
agent.display_value()
agent.display_greedy_policy()
display(batch_results)


Stage 0
[[ -42.15647321 -113.51115219 -135.95284418]
 [ -78.38006343  -60.2342063   -65.61980076]
 [-177.50787338 -168.41567307  -86.06783218]]
Stage 1
[[-122.96214145 -130.92331664 -277.14887246]
 [-205.67085673 -175.65094462 -179.69565605]
 [-250.91202203 -250.91202273 -250.91202278]]
Stage 0
  UNFIRED: engine_on 
  FIRING: engine_off
  FIRED: drop_stage
Stage 1
  UNFIRED: engine_on 
  FIRING: engine_off
  FIRED: engine_on


<xarray.Dataset>
Dimensions:             (d_2_0: 2, idx: 2520000)
Coordinates:
  * idx                 (idx) int64 0 1 2 3 ... 2519996 2519997 2519998 2519999
Dimensions without coordinates: d_2_0
Data variables:
    t                   (idx) float64 10.0 10.0 10.0 10.0 ... 10.0 10.0 10.0
    command_engine_on   (idx) bool False False False False ... True False False
    command_drop_stage  (idx) bool True True True True ... True False False True
    gii                 (idx, d_2_0) float64 -9.938e-17 -1.623 ... -1.623
    xii                 (idx, d_2_0) float64 1.064e-10 1.738e+06 ... 1.738e+06
    vii                 (idx, d_2_0) float64 2.135e-16 3.486 ... -15.65
    aii                 (idx, d_2_0) float64 -9.938e-17 -1.623 ... -1.623
    fii_thrust          (idx, d_2_0) float64 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
    theta               (idx) float64 1.571 1.571 1.571 ... 1.571 1.571 1.571
    theta_dot           (idx) int64 0 0 0 0 0 0 0 0 0 0 ... 0 0 0 0 0 0 0 0 0 0
    mass                (idx) float64 1.0 1.0 1.0 1.0 ... 1.02 1.02 1.02 1.02
    mass_dot            (idx) float64 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
    h                   (idx) float64 1.056e+03 1.056e+03 ... 926.1 926.1
    engine_on           (idx) bool False False False False ... False False False
    stage_state         (idx) int64 2 2 2 2 2 2 2 2 2 2 ... 2 2 2 2 2 2 2 2 2 2
    stage_idx           (idx) int64 1 1 1 1 1 1 1 1 1 1 ... 1 1 1 1 1 1 1 1 1 1
    gamma_i             (idx) float64 1.571 1.571 1.571 ... -1.571 -1.571 -1.571
    gamma_e             (idx) float64 1.571 1.571 1.571 ... -1.571 -1.571 -1.571
    latitude            (idx) float64 1.571 1.571 1.571 ... 1.571 1.571 1.571
    reward              (idx) float64 55.92 55.92 55.92 ... -73.86 -73.86 -73.86
    reward_integral     (idx) float64 236.7 236.7 236.7 ... -242.3 -242.3 -242.3
    delta_v             (idx) float64 -3.532 -3.532 -3.532 ... -4.931 -4.931
Attributes:
    env_surface_diameter:  1737400.0
    env_mu:                4904869500000.0
    env_stages:            (Stage(dry_mass=2, propellant_mass=0.02, specific_...
    env_initial_latitude:  1.5707963267948966
    env_initial_altitude:  1000
    env_initial_theta_e:   1.5707963267948966

## Single simulation

In [14]:
simulation.restore_states()
with time_it("simulation run"):
    agent.epsilon = 0
    result = simulation.run(100)
post_processing(result)
result

simulation run: 0.009969284998078365 [s]


<xarray.Dataset>
Dimensions:             (d_2_0: 2, t: 100)
Coordinates:
  * t                   (t) float64 0.0 0.1 0.2 0.3 0.4 ... 9.5 9.6 9.7 9.8 9.9
Dimensions without coordinates: d_2_0
Data variables:
    command_engine_on   (t) bool True False False True ... True True True True
    command_drop_stage  (t) bool False False True False ... False False False
    gii                 (t, d_2_0) float64 -9.938e-17 -1.623 ... -1.623
    xii                 (t, d_2_0) float64 1.064e-10 1.738e+06 ... 1.738e+06
    vii                 (t, d_2_0) float64 0.0 0.0 ... -9.481e-16 -15.48
    aii                 (t, d_2_0) float64 -9.938e-17 -1.623 ... -1.623
    fii_thrust          (t, d_2_0) float64 0.0 0.0 4.164e-16 6.8 ... 0.0 0.0 0.0
    theta               (t) float64 1.571 1.571 1.571 ... 1.571 1.571 1.571
    theta_dot           (t) int64 0 0 0 0 0 0 0 0 0 0 0 ... 0 0 0 0 0 0 0 0 0 0
    mass                (t) float64 2.02 2.019 2.019 1.02 ... 1.02 1.02 1.02
    mass_dot            (t) float64 0.0 -0.006932 0.0 0.0 ... 0.0 0.0 0.0 0.0
    h                   (t) float64 1e+03 1e+03 1e+03 ... 929.2 927.7 926.1
    engine_on           (t) bool False True False False ... False False False
    stage_state         (t) int64 0 1 2 0 1 2 2 2 2 2 2 ... 2 2 2 2 2 2 2 2 2 2
    stage_idx           (t) int64 0 0 0 1 1 1 1 1 1 1 1 ... 1 1 1 1 1 1 1 1 1 1
    gamma_i             (t) float64 0.0 1.571 1.571 ... -1.571 -1.571 -1.571
    gamma_e             (t) float64 0.0 1.571 1.571 ... -1.571 -1.571 -1.571
    latitude            (t) float64 1.571 1.571 1.571 ... 1.571 1.571 1.571
    reward              (t) float64 0.0 0.003107 0.01243 ... -72.32 -73.86
    reward_integral     (t) float64 0.0 1.002e-05 0.0009221 ... -227.6 -234.9
    delta_v             (t) float64 0.0 -1.799 -0.9516 ... -5.862 -5.413 -4.933
    xie                 (t, d_2_0) float64 -6.879e-27 1e+03 ... -5.434e-27 926.1
Attributes:
    env_surface_diameter:  1737400.0
    env_mu:                4904869500000.0
    env_stages:            (Stage(dry_mass=2, propellant_mass=0.02, specific_...
    env_initial_latitude:  1.5707963267948966
    env_initial_altitude:  1000
    env_initial_theta_e:   1.5707963267948966

In [15]:
agent.display_greedy_policy()

Stage 0
  UNFIRED: engine_on 
  FIRING: engine_off
  FIRED: drop_stage
Stage 1
  UNFIRED: engine_on 
  FIRING: engine_off
  FIRED: drop_stage


In [23]:
plt.figure()
result.vii.plot.line(x="t", label="command_engine_on")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [24]:
plt.figure()
result.h.plot.line(x="t", label="command_engine_on")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [16]:
plt.figure()
result.command_engine_on.plot.line(x="t", label="command_engine_on")
result.command_drop_stage.plot.line(x="t", label="command_drop_stage")
plt.legend()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [19]:
plt.figure()
(result.mass_dot * 1e3).plot.line(x="t", label="mass_dot")
(result.stage_idx + 1).plot(label="stage_idx")
(result.stage_state).plot(label="stage_state")
plt.legend()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …